In [1]:
import geneformer
from geneformer import TranscriptomeTokenizer
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import loompy

# The purpose of this notebook is to prepare our data for tokenization and then tokenize the data

In [22]:
data_path = "/u/scratch/s/schhina/geneformer_raw_data"
output_path = "/u/scratch/s/schhina/geneformer_tokenized_data"

In [24]:
adata = ad.read_h5ad("/u/scratch/s/schhina/labeled_t_cell_data.h5ad")
# adata.write_loom("/u/scratch/s/schhina/geneformer_raw_data/labeled_t_cell_data.loom")
adata

AnnData object with n_obs × n_vars = 47726 × 60725
    obs: 'stimulation', 'cd_status'

In [5]:
# A loom file is basically a transpose of a h5ad file (which is what our data is currently stored in)

ds = loompy.connect("/u/scratch/s/schhina/geneformer_raw_data/labeled_t_cell_data.loom") # Open a loom file
ds.shape

(60725, 47726)

In [6]:
# To tokenize we need the n_counts and ensembl_id fields

ds.ca['n_counts'] = [np.sum(ds[:, i]) for i in np.arange(ds.shape[1])] # Add a count column to loom file
ds.ra['ensembl_id'] = [(s.split('.')[0]) for s in ds.ra.Accession] # Add an ensembl_id row to loom file
del ds.ra['Accession'] # Remove Accession row
ds.close() # Close the loom file

In [7]:
tk = TranscriptomeTokenizer({'stimulation': 'stimulation'}, nproc=16)

In [8]:
tk.tokenize_data(data_path, output_path, "Q_data_", file_format="loom")

Tokenizing /u/scratch/s/schhina/geneformer_raw_data/labeled_t_cell_data.loom
/u/scratch/s/schhina/geneformer_raw_data/labeled_t_cell_data.loom has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Map (num_proc=16):   0%|          | 0/47726 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/47726 [00:00<?, ? examples/s]